In [ ]:
sc

In [37]:
dfhtag1 = sqlContext.read.load('stream_twt_may11.jsonl', format='json')
dfhtag1.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [50]:
#Retweeted Years
import pyspark.sql.functions as sf
timeline = dfhtag1.select('retweeted_status.created_at').dropna()
timeline = timeline.select(timeline.created_at.substr(sf.length(timeline.created_at)-25, sf.length(timeline.created_at)-27).alias("Month")\
           ,timeline.created_at.substr(sf.length(timeline.created_at)-4, sf.length(timeline.created_at)).alias("Year"))
timeline = timeline.filter(timeline.Year <= 2013)
timeline = timeline.select(sf.concat(timeline.Month, timeline.Year).alias('date')) 
#timeline.show(truncate =False, n=100)

In [51]:
import pyspark.sql.functions as sf
timeline = timeline.groupBy('date').count().orderBy('count', ascending = False)
#timeline.show(truncate=False, n=1000)

In [54]:
import pyspark.sql.functions as sf
timeline2 = dfhtag1.select('retweeted_status.text','retweeted_status.created_at').dropna()
timeline2 = timeline2.select('text',timeline2.created_at.substr(sf.length(timeline2.created_at)-25, sf.length(timeline2.created_at)-27).alias("Month")\
           ,timeline2.created_at.substr(sf.length(timeline2.created_at)-4, sf.length(timeline2.created_at)).alias("Year"))
timeline2 = timeline2.filter(timeline2.Year <= 2013)
timeline2 = timeline2.select('text',sf.concat(timeline2.Month, timeline2.Year).alias('date_'))                 
#timeline2.show(truncate =False, n=100)

In [56]:
result = timeline.join(timeline2, timeline.date == timeline2.date_)
result = result.drop('date_')

result.show(truncate=False, n=100)

+--------+-----+-------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|date    |count|text                                                                                                                                             |date_   |
+--------+-----+-------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Jun 2013|5    |#TatsuyaNakadai asking #KihachiOkamoto's widow to come on stage was a very touching and emotional moment this evening http://t.co/XwJdu9I9Wa     |Jun 2013|
|Jun 2013|5    |Dear fifth harmony. Please. Announce. The. Single. Release. Date. And. The. Name. Of. The Song. This. Week. Thank you. Simon. X                  |Jun 2013|
|Jun 2013|5    |Dear fifth harmony. Please. Announce. The. Single. Release. Date. And. The. Name. Of. The Song. This. Week. Thank you. Simon

AttributeError: 'NoneType' object has no attribute 'show'

In [34]:
result.toPandas().to_csv('timelines/may11retweet_timeline_wtxt.csv', encoding='utf-8', index=False)